In [ ]:
# Install required libraries
!pip install -q gradio seaborn


In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Rescaling, GlobalAveragePooling2D, Input, Dropout, Dense
from tensorflow.keras import layers, optimizers
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.applications import EfficientNetV2B2
import gradio as gr
import os


In [ ]:
# Dataset path and parameters
dataset_dir = "/content/drive/MyDrive/TrashType_Image_Dataset (1)"
image_size = (124, 124)
batch_size = 32
seed = 42

# ✅ Load training and validation sets
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir, validation_split=0.2, subset="training", seed=seed,
    shuffle=True, image_size=image_size, batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir, validation_split=0.2, subset="validation", seed=seed,
    shuffle=True, image_size=image_size, batch_size=batch_size
)

class_names = train_ds.class_names

In [ ]:
# Create test set from validation set
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 2)
val_dat = val_ds.skip(val_batches // 2)
test_ds_eval = test_ds.cache().prefetch(tf.data.AUTOTUNE)

# Distribution function
def count_distribution(dataset, class_names):
    total = 0
    counts = {name: 0 for name in class_names}
    for _, labels in dataset:
        for label in labels.numpy():
            counts[class_names[label]] += 1
            total += 1
    if total == 0:  # Handle empty dataset
        return {name: 0.0 for name in class_names}
    for k in counts:
        counts[k] = round((counts[k] / total) * 100, 2)
    return counts

# Bar chart
def simple_bar_plot(dist, title):
    plt.bar(dist.keys(), dist.values(), color='cornflowerblue')
    plt.title(title)
    plt.ylabel('Percentage (%)')
    plt.xticks(rotation=45)
    plt.ylim(0, 100)
    plt.tight_layout()
    plt.show()

# Plot distributions
train_dist = count_distribution(train_ds, class_names)
val_dist = count_distribution(val_ds, class_names)
test_dist = count_distribution(test_ds, class_names)
overall_dist = {k: round((train_dist[k] + val_dist[k]) / 2, 2) for k in class_names}

simple_bar_plot(train_dist, "Training Set Distribution (%)")
simple_bar_plot(val_dist, "Validation Set Distribution (%)")
simple_bar_plot(test_dist, "Test Set Distribution (%)")
simple_bar_plot(overall_dist, "Overall Class Distribution (%)")

In [ ]:
# Show sample images
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(12):
        ax = plt.subplot(4, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")


In [ ]:
# Compute class weights
class_counts = {i: 0 for i in range(len(class_names))}
all_labels = []
for _, labels in train_ds:
    for label in labels.numpy():
        class_counts[label] += 1
        all_labels.append(label)

# Get unique labels present in the training data
unique_labels = np.unique(all_labels)

class_weights_array = compute_class_weight('balanced',
                                           classes=unique_labels,
                                           y=all_labels)

# Create a dictionary with weights for all class indices
class_weights = {i: 0.0 for i in range(len(class_names))}
for i, label in enumerate(unique_labels):
    class_weights[label] = class_weights_array[i]

In [ ]:
# Data Augmentation
data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

# Base Model
base_model = EfficientNetV2B2(include_top=False, input_shape=(124, 124, 3),
                              include_preprocessing=True, weights='imagenet')
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

# Final Model
model = Sequential([
    Input(shape=(124, 124, 3)),
    data_augmentation,
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Early stopping
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=15,
                    class_weight=class_weights, callbacks=[early])


In [ ]:
# Plot training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title('Loss')
plt.legend()
plt.show()


In [ ]:
# Evaluate with confusion matrix
y_true = np.concatenate([y.numpy() for _, y in val_ds])
y_pred_probs = model.predict(val_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

cm = confusion_matrix(y_true, y_pred)
print(classification_report(y_true, y_pred, target_names=class_names, labels=np.arange(len(class_names))))

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Show predictions
for images, labels in test_ds_eval.take(1):
    predictions = model.predict(images)
    pred_labels = tf.argmax(predictions, axis=1)
    for i in range(8):
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"True: {class_names[labels[i]]}, Pred: {class_names[pred_labels[i]]}")
        plt.axis("off")
        plt.show()


In [ ]:
# Save
model.save('efficientnetv2b2.keras')

# Load
model = tf.keras.models.load_model('efficientnetv2b2.keras')


In [ ]:
# Gradio Inference Function
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

def classify_image(img):
    img = img.resize((124, 124))
    img_array = np.array(img, dtype=np.float32)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    pred_index = np.argmax(prediction)
    pred_name = class_names[pred_index]
    confidence = prediction[0][pred_index]
    return f"Predicted: {pred_name} (Confidence: {confidence:.2f})"

# Interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Garbage Classification using EfficientNetV2B2",
    description="Upload an image of garbage and the model will classify it."
)
iface.launch()
